In [6]:
from random import randint
import time
import json

In [2]:
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
options = Options()
options.headless = False
driver = webdriver.Firefox(options=options,executable_path='C:/Program Files/Mozilla Firefox/geckodriver-v0.29.0-win64/geckodriver.exe') # Add your own path here
driver.set_window_size(randint(800, 1080), randint(1024, 1920))
time.sleep(2)

In [3]:
def getMainCategoryLinks():
    driver.get('https://www.mindtools.com/')
    mainList = driver.find_element_by_class_name('hp-toolkit').find_elements_by_tag_name('li')
    mainListEls = [li.find_element_by_tag_name('a') for li in mainList]
    mainLinks = [a.get_attribute('href')for a in mainListEls]
    catNames = [a.find_element_by_class_name('skill-area').text for a in mainListEls]
    final = list()
    for link, name in zip(mainLinks,catNames):
        g = dict()
        g['category-name'] = name
        g['category-link'] = link
        g['category-content'] = list()
        final.append(g)
    return final

In [ ]:
final = getMainCategoryLinks()
with open("metadata/main-cats.json", mode='w') as f:
    f.write(json.dumps(final))

In [5]:
def getCategoryArticleLinks(cat):
    try:
        driver.get(cat['category-link'])
        time.sleep(randint(40,90))
        articleTags = driver.find_element_by_class_name('bycat').find_elements_by_class_name('collection')
        articleCats = driver.find_element_by_class_name('bycat').find_elements_by_class_name('cat-subheader')
        subCategories = list()
        articleCats[-1].click() # so that the first element is clickable
        
        for i,el in enumerate(articleTags):
            subCat = dict()
            subCat['sub-category-name'] = (articleCats[i].text).split('(')[0]
            articleCats[i].click()
            subCat['sub-category-content'] = list()
            for a in el.find_elements_by_tag_name('a'): 
                article = {}
                article['link'] = a.get_attribute('href')
                if a.find_element_by_class_name('subheading').text:
                    article['title'] = a.find_element_by_class_name('heading').text + '- ' +  a.find_element_by_class_name('subheading').text
                else:
                    article['title'] = a.find_element_by_class_name('heading').text
                print(subCat['sub-category-name'],'-',article['title'])
                article['content'] = ''
                subCat['sub-category-content'].append(article)
            subCategories.append(subCat)
            print('----------------------------------------')
        print('Returning subcategory', cat['category-link'],'\n')
        with open(f"metadata/{cat['category-name']}-sub.json", mode='w') as f:
            f.write(json.dumps(subCategories))
        return subCategories
    except Exception as e:
        print(cat['category-name'],'Error while fetching article links', str(e))
        return None

In [ ]:
for cat in final:
    subcats = getCategoryArticleLinks(cat)
    cat['category-content']= subcats